In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!python -V

Python 3.11.9


In [3]:
import json
import pickle

import requests
import pandas as pd

In [ ]:
with open('homework/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
year = 2023
month = 3
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

# Q1
Standard deviation of the predicted trip duration

In [8]:
y_pred.std()

6.247488852238703

# Q2
Size of dataframe file with predictions: 66MB

In [9]:
df["ride_id"] = f"{year:04d}/{month:02d}_" + df.index.astype("str")

In [10]:
output_file = f"output/yelow_taxi_{year:04d}-{month:02d}.parquet"

df_result = pd.DataFrame()
df_result["ride_id"] = df["ride_id"].copy()
df_result["prediction"] = y_pred

df_result.to_parquet(
    output_file,
    engine="pyarrow",
    compression=None,
    index=False
)

# Q3

`!jupyter nbconvert --to script starter.ipynb`

In [11]:
# !jupyter nbconvert --to script homework/starter.ipynb

# Q4

`pipenv install scikit-learn==1.5.0 pandas --python=3.10.13`

scikit-learn hash:
`sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c`

# Q5

mean predicted duration: 14.29

In [ ]:
!python homework/starter.py --month 4 --year 2023

# Q6

In [13]:
url = "http://0.0.0.0:9696/predict"
data = {
    "year": 2023,
    "month": 5
}
response = requests.post(url, json=data)

In [14]:
json.loads(response.content)["mean_predicted_duration"]

0.19174419265916945